In [ ]:
# pip install giotto-tda
# pip install gudhi

In [6]:
import pandas as pd
import numpy as np
import matplotlib as plt
from gtda.time_series import TakensEmbedding

time_series = np.log(pd.read_csv("SP500.csv", header=None).to_numpy().squeeze())

In [ ]:
def TakenEmbedding(time_series, embedded_dimension, time_delay):
    L = len(time_series)
    N = embedded_dimension
    d = time_delay
    embedding = np.array([])
    for i in range(L - (N - 1) * d):
        window = time_series[i:]
        window = window[::d]
        embedding = [:N]
        embedding = np.append(embedding, )